# Here i want to change data from Chess_data_basic to other format explained later

So here i want to get data in format similar to that: 

columns = ( date, amount of mathes played in that date,ammont of matches played day earllier , amount of matches plyed for previous mounth, ranikng points fluctuation (day vs day before), ranking after last match that day, mean oponent vs my ranking diff that day ) and maybe some other columns that i will consider as important for ranking changes prediction. 

And all of those for different times control (blitz, rapid, bullet)

# To Be Continued

In [2]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv("Chess_data_basic.csv", index_col = "Unnamed: 0")

In [17]:
df

,id,rated,variant,speed,createdAt,lastMoveAt,status,winner,moves,clock,white_player_name,white_ranking,black_player_name,black_ranking
0,uxE9ud30,True,standard,blitz,2023-06-01 18:22:19.034000+02:00,2023-06-01 18:26:26.523000+02:00,resign,black,30,300,zajc1,1750,EvilSaintPL,1763
1,DRiFLKVy,True,standard,blitz,2023-06-01 18:07:38.533000+02:00,2023-06-01 18:17:22.238000+02:00,outoftime,white,69,300,lc1408,1759,EvilSaintPL,1768
2,nSkSOyKk,True,standard,blitz,2023-06-01 16:44:58.061000+02:00,2023-06-01 16:50:56.745000+02:00,mate,white,30,300,EvilSaintPL,1763,Oncedead,1742
3,ypRGupyw,True,standard,blitz,2023-06-01 03:37:21.296000+02:00,2023-06-01 03:47:18.336000+02:00,outoftime,black,36,300,Jorjao,1687,EvilSaintPL,1758
4,xc0m55P3,True,standard,blitz,2023-06-01 03:28:56.495000+02:00,2023-06-01 03:37:12.280000+02:00,mate,black,51,300,EvilSaintPL,1765,Jorjao,1680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5657,dYnYtNFc,True,standard,blitz,2019-11-09 20:53:37.672000+01:00,2019-11-09 20:59:37.804000+01:00,outoftime,white,28,180,EvilSaintPL,1206,zfred12,1311
5658,HLTrX7cv,True,standard,blitz,2019-11-09 20:51:04.051000+01:00,2019-11-09 20:53:25.908000+01:00,mate,white,16,180,zfred12,1305,EvilSaintPL,1272
5659,enP16ebg,True,standard,blitz,2019-11-09 20:46:10.220000+01:00,2019-11-09 20:49:53.831000+01:00,mate,black,23,180,Mokrakifla,1061,EvilSaintPL,1216
5660,8cRwRfdY,True,standard,blitz,2019-11-09 20:45:24.715000+01:00,2019-11-09 20:46:02.537000+01:00,resign,white,6,180,NecroCosmos98,1347,EvilSaintPL,1326
